In [ ]:
%pip install git+https://github.com/Open-Dataplatform/utils-databricks.git@v0.2.1

In [ ]:
import os
from pyspark.sql.functions import col
import pyspark.sql.functions as F

from custom_utils.dp_storage.connector import mount, unmount_if_prod
from custom_utils.dp_storage.writer import merge_and_upload

# Transformation

Setup

In [ ]:
mount_point = mount(dbutils)

# Create parameter widgets
dbutils.widgets.text('IngressIdentifier', 'TODO: default_value', '')
dbutils.widgets.text('EgressIdentifier', 'TODO: default_value', '')
dbutils.widgets.text('DateKeyName', 'TODO: default_value', '')
dbutils.widgets.text('DateFormat', 'TODO: default_value', '')
dbutils.widgets.text('TimeResolution', 'TODO: default_value', '')
dbutils.widgets.text('FilePath', 'TODO: default_value', '')
dbutils.widgets.text('FullLoad', 'TODO: default_value', '')

# Get parameters
ingress_identifier = dbutils.widgets.get('IngressIdentifier')
egress_identifier = dbutils.widgets.get('EgressIdentifier')
date_key_name = dbutils.widgets.get('DateKeyName')
date_format = dbutils.widgets.get('DateFormat')
time_resolution = dbutils.widgets.get('TimeResolution')
file_path = dbutils.widgets.get('FilePath')
is_full_load = dbutils.widgets.get('FullLoad')

# Define key columns
index_columns = [date_key_name, 'TODO: Add all key columns']

## Read
Reads data from storage

In [ ]:
source_path = os.path.join(mount_point, file_path)

df = spark.read.parquet(source_path)
# df = spark.read.option("delimiter", ",").csv(source_path, header=True)

In [ ]:
df.show(3)

## Transform

Transform the data here. Follow this style guide: https://github.com/palantir/pyspark-style-guide

## Merge and upload

In [ ]:
merge_and_upload(
    df,
    egress_identifier,
    date_key_name,
    index_columns,
    time_resolution,
    mount_point,
    spark
)

In [ ]:
# Always keep this at the end of your notebook
unmount_if_prod(mount_point, dbutils)